In [ ]:
### Cosc4337 Fall 2022 Rizk - HW5

In [ ]:
# Dosbol Aliev
# MyUH 1867424
# Data SCience II

In [ ]:
# For this homework, I have used Google Collab, so It was no need to install Libraries

In [ ]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

In [ ]:
torch.manual_seed(111)

In [ ]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")

In [ ]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

In [ ]:
train_set = torchvision.datasets.MNIST(
    root=".", train=True, download=True, transform=transform
)

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

In [ ]:
real_samples, mnist_labels = next(iter(train_loader))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(real_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.5), # Making DropOut Rate 50%
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5), # Making DropOut Rate 50%
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5), # Making DropOut Rate 50%
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output
discriminator=Discriminator().to(device=device)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 1, 28, 28)
        return output
generator=Generator().to(device=device)

In [ ]:
lr = 1 * (10)**(-4)
num_epochs = 50 # Making number of Epochs 50 because in pdf file also 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [ ]:
for epoch in range(num_epochs):
    for n,(real_samples,mnist_labels) in enumerate(train_loader):
        
        # Data for training the discriminator
        real_samples=real_samples.to(device=device)
        real_sample_labels=torch.ones((batch_size,1)).to(device=device)
        
        
        latent_space_samples=torch.randn((batch_size,100)).to(device=device)
        
        
        generated_samples=generator(latent_space_samples)
        

        generated_sample_labels=torch.zeros((batch_size,1)).to(device=device)
        
        
        all_samples=torch.cat((real_samples,generated_samples))
        all_sample_labels=torch.cat((real_sample_labels,generated_sample_labels))
        
        # Training the discriminator
        
        
        discriminator.zero_grad()   
        output_discriminator=discriminator(all_samples)
        loss_discriminator=loss_function(output_discriminator,all_sample_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()
        
        
        
        latent_space_samples=torch.randn((batch_size,100)).to(device=device)
        
        
        
        generator.zero_grad()
        generated_samples=generator(latent_space_samples)
        output_discriminator_generated=discriminator(generated_samples)
        loss_generator=loss_function(output_discriminator_generated,real_sample_labels)
        loss_generator.backward()
        optimizer_generator.step()
        
         # Show loss
        if n==batch_size-1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")
            plt.show(epoch,loss_discriminator)
            plt.show(epoch,loss_generator)
        

In [ ]:
latent_space_samples = torch.randn(batch_size, 100).to(device=device)
generated_samples = generator(latent_space_samples)

In [ ]:
# Make sure your output is something like what is below and you should be good.

In [ ]:
generated_samples = generated_samples.cpu().detach()
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(generated_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])